## <span style="color:grey">Initialization</span>

In [1]:
try:
    mode
except Exception as e:
    %run "KiteConnect_Library.ipynb"  
    logging.debug(e)
    mode = "Algo"
    papertrade = True

tradelogger.setLevel(logging.DEBUG)

In [2]:
tradelogger.info("Let's do trading")

##  <span style="color:green">Trading Strategy </span>

In [321]:
# ====== Tradescript Wrapper =======
# Methods
REF = lambda df, i: df.shift(i)
TREND_UP = lambda a,b: ROC(a, b) >= 0.1
TREND_DOWN = lambda a,b: ROC(a, b) <= -0.1
CROSSOVER = lambda a, b: (REF(a,1)<=REF(b,1)) & (a > b)
isblackout = True
blackout = lambda a,x,y='11:30:00',z='12:30:00': a[isblackout & (a.index >= x) & (a.index < x+' '+y) | (a.index > x+' '+z)]


from talib import MIN, MAX, CDLLONGLEGGEDDOJI, CDLMARUBOZU, CDLLONGLINE, CDLDARKCLOUDCOVER, CDLPIERCING, CDLBELTHOLD, CDLCLOSINGMARUBOZU

class algoTrade:
   
    def __init__(s, price):
        logging.debug(price.tail(10))
        s.OPEN = price['open']
        s.CLOSE = price['close']
        s.HIGH = price['high']
        s.LOW = price['low']
        s.BBT, s.BBM, s.BBB = BBANDS( s.CLOSE, 20,1.75,1.75,0)
        #s.BBT2, s.BBM2, s.BBB2 = BBANDS( s.CLOSE, 20,3,3,0)
        s.macd, s.macdsignal, s.macdhist = MACDEXT(s.CLOSE, fastperiod=12, slowperiod=26, signalperiod=9,  fastmatype=1, slowmatype=1,signalmatype=1)
        s.SD = STDDEV(s.CLOSE)
        s.fastk, s.fastd = STOCHF(s.HIGH, s.LOW, s.CLOSE)
        s.iRSI = RSI(s.CLOSE, timeperiod=20)
        s.STATE = ""
        s.profit = 0
        s.stoploss = 0
        s.openpositions = 0
        s.target = 0
        
        s.haOPEN  = (s.OPEN.shift(1) + s.CLOSE.shift(1))/2
        s.haHIGH  = pd.DataFrame([s.HIGH,s.OPEN,s.CLOSE]).max(axis = 0, skipna = True)
        s.haLOW   = pd.DataFrame([s.HIGH,s.LOW,s.CLOSE]).min(axis = 0, skipna = True)
        s.haCLOSE = (s.OPEN+s.HIGH+s.LOW+s.CLOSE)/4
    
    # Long Strategies
    def long_bb(s):
        temp_df = 0
        temp_df = temp_df | (REF(s.fastk,1) <=20)&(s.fastk>20)
        temp_df = temp_df | CROSSOVER(s.fastk, s.fastd)
        temp_df = temp_df | (REF(s.iRSI,1) <=30)&(s.iRSI>30)
        temp_df = temp_df & CROSSOVER(s.CLOSE, s.BBM) | CROSSOVER(s.CLOSE, s.BBB)
        return pd.DataFrame(  temp_df, columns=["buy"] )
     
    # Short Strategies
    def short_bb(s):
        temp_df = 0
        temp_df = temp_df | (REF(s.fastk,1) >=80)&(s.fastk<80) & CROSSOVER(s.fastd, s.fastk)
        #temp_df = temp_df | CROSSOVER(fastd, fastk)
        temp_df = temp_df | (REF(s.iRSI,1) >=70)&(s.iRSI<70)
        temp_df = temp_df & CROSSOVER(s.BBT, s.CLOSE) | CROSSOVER(s.BBM, s.CLOSE)
        return pd.DataFrame( temp_df, columns=["sell"])
    
    
    def long_pa(s):
        temp_df = 0
        temp_df =   (CDLENGULFING(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100)
        temp_df = temp_df | (CDLHARAMI(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100)
        temp_df = temp_df | (CDLPIERCING(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100)
        temp_df = temp_df | (CDLHAMMER(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100)
        temp_df = temp_df | (CDLDOJISTAR(s.OPEN, s.HIGH, s.LOW, s.CLOSE) != 0)
        temp_df = temp_df | ((CDLMORNINGSTAR(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100) & (CDLMORNINGDOJISTAR(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100))
        temp_df = temp_df | (CDLMARUBOZU(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100) | (CDLCLOSINGMARUBOZU(s.OPEN, s.HIGH, s.LOW, s.CLOSE) ==100)
        temp_df = temp_df | (CDLBELTHOLD(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100)
        
        temp_df = REF(TREND_DOWN(s.CLOSE,4),1) & temp_df
        
        return pd.DataFrame(temp_df, columns=["buy"])
    def short_pa(s):
        temp_df = 0
        
        temp_df =   (CDLENGULFING(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDLHARAMI(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDLDARKCLOUDCOVER(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDLDOJISTAR(s.OPEN, s.HIGH, s.LOW, s.CLOSE) != 0)
        temp_df = temp_df | (CDLHANGINGMAN(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | ((CDLEVENINGSTAR(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100) & (CDLEVENINGDOJISTAR(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100))
        temp_df = temp_df | (CDLMARUBOZU(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100) | (CDLCLOSINGMARUBOZU(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDLBELTHOLD(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        
        temp_df = REF(TREND_UP(s.CLOSE,4),1) & temp_df
        
        return pd.DataFrame( temp_df , columns=["sell"])
    
    def long_breakout(s):
        temp_df = s.haOPEN >= MAX(s.BBT.shift(2), timeperiod=5)
        temp_df = temp_df & (s.haHIGH > s.haHIGH.shift(1))
       # print(pd.DataFrame({"high":s.HIGH, "max":MAX(s.HIGH, timeperiod=15), "t1": s.HIGH>= MAX(s.HIGH, timeperiod=15)
       #                     ,"low":s.LOW, "min":MIN(s.LOW, timeperiod=15)}))
        return pd.DataFrame( temp_df , columns=["buy"])
    
    def short_breakout(s):
        temp_df = s.haCLOSE <= MIN(s.BBB.shift(2), timeperiod=5)
        temp_df = temp_df & (s.haLOW < s.haLOW.shift(1))
        return pd.DataFrame( temp_df , columns=["sell"])
    
  
    def tradeDecision(s):
        global isblackout
        isblackout = True
        s.buy  = s.long_bb() & s.long_pa() #s.long_breakout() #
        s.sell = s.short_bb() #s.short_breakout() #& self.short_pa()
        
        s.buy['low'] = s.LOW
        s.sell['low'] = s.LOW
        s.buy['close'] = -1 * s.CLOSE
        s.sell['close'] = s.CLOSE

        s.buy = s.buy[s.buy['buy']]
        s.sell = s.sell[s.sell['sell']]
        
        if s.buy.empty == False:
            s.buy = blackout(s.buy,a.buy.index[-1].date().strftime("%Y-%m-%d"))
        if s.sell.empty == False:
            s.sell = blackout(s.sell, a.sell.index[-1].date().strftime("%Y-%m-%d"))
        
        #temp_df = pd.DataFrame(buy.close)
        
        s.buysell = pd.concat([s.buy.close, s.sell.close]).sort_index()
        s.buysell = s.buysell[s.buysell.index>s.buysell.index[-1].date().strftime("%Y-%m-%d")]
        #temp_df.append(buy.close)
        #temp_df = temp_df.append(sell.close)
        #logging.debug(s.buysell)
        return (s.buy,s.sell)
    
    def payoff(s):
        global isblackout
        payoff_df = pd.DataFrame(s.buysell)
        openpositions = 0
        squareoff = True
                    
        morningCutOff = s.CLOSE.index[-1].strftime("%Y-%m-%d 11:30:00")
        for i, p in payoff_df.iterrows():
            if squareoff and isblackout and i >= payoff_df.index[-1].replace(hour=11, minute=30, second=0, microsecond=0):
                if openpositions > 0:
                    s.profit = s.profit + a.CLOSE.loc[morningCutOff]
                    openpositions = 0
                    #print("SELL Squareoff:[11:30]",openpositions, a.CLOSE.loc[morningCutOff], round(s.profit[0],2))
                elif  openpositions < 0:
                    s.profit = s.profit - a.CLOSE.loc[morningCutOff]
                    openpositions = 0
                    #print("BUY Squareoff:[11:30]",openpositions, a.CLOSE.loc[morningCutOff], round(s.profit[0],2))
                    
                s.STATE = ""
                squareoff = False
                
            if not squareoff and  i <= payoff_df.index[-1].replace(hour=12, minute=30, second=0, microsecond=0):
                continue
            
                
            if s.STATE != "SELL" and p.values < 0:
                s.STATE = "SELL"
                s.profit = s.profit + p.values
                openpositions = openpositions + 1
                #print("BUY:",i,openpositions, p.close, round(s.profit[0],2))
            if s.STATE != "BUY" and p.values > 0:
                s.profit = s.profit + p.values
                s.STATE = "BUY"
                openpositions = openpositions - 1
                #print("SELL:",i,openpositions, p.close, round(s.profit[0],2))
         
        if isblackout:
            close = a.CLOSE.loc[s.CLOSE.index[-1].strftime("%Y-%m-%d 15:00:00")]
        else:
            close = a.CLOSE[-1]
        
        if openpositions > 0:
            s.profit = s.profit + close
            openpositions = 0
            #print("SELL Squareoff:[15:00]",openpositions,close,round(s.profit[0],2))
        elif openpositions < 0:
            s.profit = s.profit - close
            openpositions = 0
            #print("BUY Squareoff:[15:00]",openpositions,close, round(s.profit[0],2))
            
        return(s.profit)
    
    def plot(s, symbol, noofcandles=120, ha=False):
        init_notebook_mode(connected=True)
        fig = tools.make_subplots(rows=3, cols=1, shared_xaxes=True, row_width=[1,1,3], vertical_spacing = 0.05)
        fig['layout']['xaxis'] = dict(rangeslider = dict(visible=False), side="bottom") #, range=[xMin,xMax])
        fig['layout'].update(height=750, plot_bgcolor='rgba(0,0,0,0)', title="Charts for "+symbol)
        
        fig['layout']['yaxis']['anchor'] = 'x'
        fig['layout']['yaxis']['side'] = 'right'

        fig['layout']['xaxis']['rangeselector'] = dict(
                    buttons=list([dict(count=1, label='1h', step='hour', stepmode='backward'),
                                  dict(count=2, label='2h', step='hour', stepmode='backward'),
                                  dict(count=3, label='3h', step='hour', stepmode='backward'),
                                  dict(count=6, label='4h', step='hour', stepmode='backward'),
                                  dict(count=6, label='1d', step='hour', stepmode='backward'),
                                  dict(step='all')]))
        
        yMin = s.LOW.iloc[-1*noofcandles:-1].min()*0.99
        yMax = s.HIGH.iloc[-1*noofcandles:-1].max()*1.001

        xMin = s.CLOSE.index[-1*noofcandles]
        xMax = s.CLOSE.index[-1]
        fig['layout']['yaxis']['range'] = [yMin, yMax]
        fig['layout']['xaxis']['range'] = [xMin, xMax]
                
        traceBuy=go.Scatter(x=s.buy.index.astype('str'), y=(s.buy.low*0.994), name='BUY', mode="markers",marker = dict(color = 'rgb(119, 221, 119)',size = 20, symbol='triangle-up'),showlegend=False)
        traceSell=go.Scatter(x=s.sell.index.astype('str'), y=(s.sell.low*0.994), name='SELL', mode="markers",marker = dict(color = 'rgb(255, 0, 0)',size = 20, symbol='triangle-up'),showlegend=False)
       
        if ha:
            trace = go.Candlestick(x=s.CLOSE.index.astype('str'), open=s.haOPEN, high=s.haHIGH, low=s.haLOW, close=s.haCLOSE, name="Heikin Ashi", showlegend=False)
        else:
            trace = go.Candlestick(x=s.CLOSE.index.astype('str'), open=s.OPEN, high=s.HIGH, low=s.LOW, close=s.CLOSE, name="Candlestick", showlegend=False)
            
        traceMACD = go.Scatter(x=s.CLOSE.index.astype('str'), y=s.macd, name='MACD', line=dict(color='black'),showlegend=False)
        traceMACDSignal = go.Scatter(x=s.CLOSE.index.astype('str'), y=s.macdsignal, name='MACD signal', line=dict(color='red'),showlegend=False)
        traceMACDHist = go.Bar(x=s.CLOSE.index.astype('str'), y=s.macdhist, name='MACD Hist', marker=dict(color="grey"),showlegend=False)
        traceSK  = go.Scatter(x=s.CLOSE.index.astype('str'), y=s.fastk, name='%K', line=dict(color='black'), yaxis='y3',showlegend=False)
        traceSD  = go.Scatter(x=s.CLOSE.index.astype('str'), y=s.fastd, name='%D', line=dict(color='red'),showlegend=False)
        traceBBT = go.Scatter(x=s.BBT.index.astype('str'), y=s.BBT, name='BB_up',  line=dict(color='lightgrey'),showlegend=False)
        traceBBB = go.Scatter(x=s.BBB.index.astype('str'), y=s.BBB, name='BB_low',  line=dict(color='lightgrey'), fill = 'tonexty', fillcolor="rgba(0,40,100,0.02)",showlegend=False)
        traceBBM = go.Scatter(x=s.BBM.index.astype('str'), y=s.BBM, name='BB_mid',  line=dict(color='lightgrey'), fill = 'tonexty', fillcolor="rgba(0,40,100,0.02)",showlegend=False)
         
        fig.append_trace(traceBBT, 1, 1)
        fig.append_trace(traceBBB, 1, 1)
        fig.append_trace(traceBBM, 1, 1)
        fig.append_trace(traceBuy, 1, 1)
        fig.append_trace(traceSell, 1, 1)
    
        fig.append_trace(traceSK, 2, 1)
        fig.append_trace(traceSD, 2, 1)
        
        fig['layout']['yaxis2']['title']="Stochastics"
        fig['layout']['yaxis2']['side']="right"
        fig['layout']['yaxis2']['anchor']="x"
        fig['layout']['yaxis2']['tickvals']=[20,80,30,70]
        
        fig.append_trace(traceMACD, 3, 1)
        fig.append_trace(traceMACDSignal, 3, 1)
        fig.append_trace(traceMACDHist, 3, 1)
        fig.append_trace(trace, 1, 1)
        
        fig['layout']['yaxis3']['anchor']="x"
        fig['layout']['yaxis3']['side']="right"
        fig['layout']['yaxis3']['title']="MACD"
        fig['layout']['yaxis1']['title']="Candlestick"
        
        iplot(fig, filename="plot/"+x+".html")
        
    # do not set timestamp for live trading
    def tradeBot(s, symbol, timestamp=dt.datetime.now().replace(second=0, microsecond=0)):
        
        timeNow = timestamp        
        delT1 = timeNow - dt.timedelta(minutes = 1)
        delT2 = timeNow - dt.timedelta(minutes = 2)
        
        #atr = ATR(HIGH, LOW, CLOSE)[-1]
        s.atr = s.CLOSE[-1]*0.025

        price = s.CLOSE[-1]
        trigger = s.CLOSE[-1]
        longSL = toTick(s.CLOSE[-1] - s.atr)
        shortSL = toTick(s.CLOSE[-1]+s.atr)
        longTarget = toTick(s.CLOSE[-1] + s.atr) 
        shortTarget = toTick(s.CLOSE[-1]- s.atr)

        check_trade = lambda x, dt1, dt2: True if (x.shape[0] > 0) and ((x.index[-1]==dt1) or (x.index[-1]==dt2)) else False
        print_trade = lambda x: logging.debug(x) if (x.empty != True) else False
        
        if papertrade == True and STATE != "":
            if STATE == "SELL":
                if s.LOW[-1]<=s.stoploss:
                    temp_price = s.stoploss
                    s.STATE = ""
                    s.openpositions = s.openpositions - 1
                    tradelogger.info("Squareoff Stoploss: sell: "+str(temp_price))
                    tradelogger.debug(openpositions)
                    return temp_price
                elif HIGH[-1]>=target:
                    temp_price = s.target
                    s.STATE = ""
                    s.openpositions = s.openpositions - 1
                    tradelogger.info("Squareoff Target: sell: "+str(temp_price))
                    tradelogger.debug(openpositions)
                    return temp_price
            elif STATE == "BUY":
                if HIGH[-1]>=stoploss:
                    temp_price = -1 * s.stoploss
                    s.STATE = ""
                    s.openpositions = s.openpositions + 1
                    tradelogger.info("Squareoff Stoploss: buy: "+str(temp_price))
                    tradelogger.debug(openpositions)
                    return temp_price
                elif LOW[-1]<=target:
                    temp_price = -1 * s.target
                    s.STATE = ""
                    s.openpositions = s.openpositions + 1
                    tradelogger.info("Squareoff Target: buy: "+str(temp_price))
                    tradelogger.debug(openpositions)
                    return temp_price
                
            
        
        buy, sell = s.tradeDecision()
        
        tradelogger.debug("STATE:" + s.STATE)
        tradelogger.debug(buy)
        tradelogger.debug(sell)
        
        
        tradelogger.debug(timeNow)
        tradelogger.debug(delT1)
        tradelogger.debug(delT2)
        
        # BUY in BUY state, SELL in STATE state
        if check_trade(buy, delT1, delT2) and (STATE != "SELL"):            
            tradelogger.info("buy:"+str(buy.index[-1]))
            s.openpositions = s.openpositions + 1
            s.stoploss = longSL
            target = longTarget
            tradelogger.debug(s.openpositions)
            
            qty = 1
            if s.openpositions != 0:
                qty = 2
                s.openpositions = s.openpositions + 1
            #tradelogger.info(atr)
            ##### Order Placement #####
            buy_bo(symbol, price, trigger, longSL, longTarget, quantity=qty)
            #buy_slm(symbol, price, stoploss, quantity=1)
            s.STATE = "SELL"
            return -1 * price * qty
            
        elif check_trade(sell, delT1, delT2) and (STATE!="BUY"):                
            tradelogger.info("sell:"+str(sell.index[-1]))
            openpositions = openpositions - 1
            stoploss = shortSL
            target = shortTarget
            tradelogger.debug(s.openpositions)
            #tradelogger.info(atr)
            ##### Order Placement #####
            qty = 1
            if s.openpositions != 0:
                qty = 2
                s.openpositions = s.openpositions - 1
            sell_bo(symbol, price, trigger, shortSL, shortTarget, quantity=qty)
            #sell_slm(symbol, price, stoploss, quantity=1)
            s.STATE = "BUY"
            return price * qty
        
        return 0


if mode == "Algo" and True:
    x = 'BHARTIARTL'
    
    toDate = dt.datetime(2019,4,27,15,15)
    fromDate =   getFromDate(toDate)
    #logger.setLevel(logging.DEBUG)
    portfolio = pd.DataFrame([x])
    papertrade = True
    
    portfolio = pd.DataFrame(downloadlist)
    for i,y in portfolio.iterrows():
        #print(y[0])
    
        temp_data = getData(y[0], fromDate, toDate, "minute")

        a = algoTrade(temp_data)

        buy, sell = a.tradeDecision()


        print(y[0],round(a.payoff()[0],2), str(round(a.payoff()[0]/a.CLOSE[-1]*100,2))+"%" )
    #a.plot(x, 360, False)
    #print(a.buysell)
    
    

ADANIPORTS -2.1 -0.23%
ASIANPAINT 5.7 -0.32%
AXISBANK -4.75 -1.24%
BAJAJ-AUTO 25.55 1.68%
BAJFINANCE -27.85 -2.47%
BAJAJFINSV -22.7 -0.59%
BPCL -0.4 0.54%
BHARTIARTL 8.8 5.41%
INFRATEL -1.5 -1.47%
BRITANNIA -10.35 -0.7%
CIPLA -8.2 -2.0%
COALINDIA 2.9 1.34%
DRREDDY 9.0 -0.53%
EICHERMOT 2.15 -0.18%
GAIL -5.15 -1.93%
GRASIM 12.1 0.93%
HCLTECH 3.15 -0.23%
HDFCBANK -0.2 -0.1%
HEROMOTOCO -14.0 -1.08%
HINDALCO -2.05 -0.67%
HINDUNILVR 8.05 0.8%
HDFC -18.35 -2.25%
ITC 1.1 0.28%
ICICIBANK -1.7 -0.27%
IBULHSGFIN -8.1 -2.2%
IOC 3.75 3.62%
INDUSINDBK -24.25 -2.86%
INFY -6.15 -0.85%
JSWSTEEL -8.8 -3.47%
KOTAKBANK -5.9 -1.24%
LT 4.3 0.87%
M&M 3.7 1.12%
MARUTI -38.5 -0.57%
NTPC -0.5 -0.74%
ONGC -0.7 -0.83%
POWERGRID 1.35 1.28%
RELIANCE 0.95 -0.27%
SBIN -3.65 -2.23%
SUNPHARMA 2.1 0.95%
TCS -31.15 -2.1%
TATAMOTORS -2.0 -1.86%
TATASTEEL -10.7 -2.12%
TECHM 2.95 0.59%
TITAN 13.25 1.96%
UPL -2.05 -0.42%
ULTRACEMCO -23.7 -1.02%
VEDL -0.45 -0.53%
WIPRO 0.5 0.41%
YESBANK 1.05 1.62%
ZEEL 0.05 -0.37%


In [252]:
a.buy.index[-1].date()

datetime.date(2019, 4, 22)

In [260]:
a.buy.index[-1].replace(hour=0, minute=0, second=0, microsecond=0)

Timestamp('2019-04-22 00:00:00')

In [8]:
tradelogger.setLevel(logging.INFO)
if mode == "Algo":
    #portfolio = downloadlist
    portfolio = ['BHARTIARTL']
    toDate = dt.datetime(2019,4,24,15,15)
    fromDate = getFromDate(toDate)
    STATE = ""
    stoploss = 0
    target = 0
    openpositions = 0
    portfolio = pd.DataFrame(portfolio)
    papertrade = True
    backtest(portfolio, fromDate, toDate)

BHARTIARTL: -5.8, (-1.7918 %)


- Lunch Time: 11:00 am to 2:00 pm => Choppy Market
- Bollinger band ZONE: for breakout

Choppy market Trading:
- Identify stocks with highest SD and trade using bollinger

Trending Market Trading:
- MACD or price action trading based on candle stick


No new open position after 3:00. Only close open positions

before 9:15: Go long based on one day candle
9:15 - 11:30: MACD or Stochastic+BB: One trigger, and atleast one confirmation
11:30 - 2:00: No trade untill and unless there is a golden cross or death cross
2:00 - 2:45: Go long if price is smaller than VWAP, Go short if price is higher than VWAP
2:00 - 2:45: Go long or short only if price crosses SMA(200)


In [12]:
import sys
from multiprocessing import Process
import time

def f(name):
    print('hello', name)
    logger.debug("Hello")

if __name__ == '__main__':
    logger.debug("hello")
    p = Process(target=f, args=('bob',))
    p.start()
    p.join()
    sys.stdout.flush()
    time.sleep(5)

In [26]:
import test
from multiprocessing import Pool

if __name__ ==  '__main__': #this is the only additional line
    num_processors = 3
    p=Pool(processes = num_processors)
    output = p.map(test.worker,[i for i in range(0,3)])
    print(output)

[0, 1, 4]


In [21]:
class A:
    varA = 15
    def disp2(self):
        print("A: ",self.varA)
    pass

class B(A):
    def disp(self):
        self.disp2()
        print("B :",self.varA)
        
b = B()
b.disp()

A:  15
B : 15
